**The same task as Lab 1 except using `RNNs`**

**Watch out!<br>
Preproceesing here is a a bit different**

**Dataset**
labeled dataset collected from twitter

**Objective**
classify tweets containing hate speech from other tweets. <br>
0 -> no hate speech <br>
1 -> contains hate speech <br>

**Total Estimated Time = 60-90 Mins**

> **Load the `clean data` preprocessed in `Lab 1`, then handle it to be used with `RNNs`**

# Import Libraries

In [1]:
import pandas as pd
import os
import urllib

In [2]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense, Bidirectional, TextVectorization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras import Input
import numpy as np

In [3]:
SEED = 42

# Load Dataset

In [4]:
# Check whether we're using google colab
using_colab = None
CWD = os.getcwd()
if os.path.exists('/content'):
  using_colab = True
else:
  using_colab = False
  PWD = os.path.dirname(CWD)

here


In [ ]:
# # Download the dataset
# !gdown https://drive.google.com/u/0/uc?id=1Ir9Rwy1iBOfDfIoaQWDGgFT0W-YSNmot&export=download

In [5]:
if using_colab:  #Check if using Colab
    data_folder_path = CWD #+ "/data/"
else:
    data_folder_path = PWD + "data/"
train_csv_file_path = data_folder_path + "arabic_dialects_train.csv"
test_csv_file_path = data_folder_path + "arabic_dialects_test.csv"

In [6]:
train_url = 'https://github.com/AmgadHasan/arabic-dialect-detection/raw/main/data/arabic_dialects_train.csv'
test_url = 'https://github.com/AmgadHasan/arabic-dialect-detection/raw/main/data/arabic_dialects_test.csv'

In [7]:
if not os.path.isfile(train_csv_file_path):
    # If the file doesn't exist, download it from the URL
    print("Downloading file...")
    urllib.request.urlretrieve(train_url, train_csv_file_path)
    print("File downloaded.")
else:
    print("File exists.")

File downloaded.


In [8]:
if not os.path.isfile(test_csv_file_path):
    # If the file doesn't exist, download it from the URL
    print("Downloading file...")
    urllib.request.urlretrieve(test_url, test_csv_file_path)
    print("File downloaded.")
else:
    print("File exists.")

File downloaded.


In [9]:
df = pd.read_csv(train_csv_file_path, lineterminator='\n')
pd.set_option('display.max_colwidth', 0)
df.head()

,tweet,label
0,انتي جافية والا الغلا ماتبيسؤالموسيقي,LY
1,باركوا لجوجو اجاها عريس,LB
2,لابأس عليك يالزينه ان شاء الله هذا حد السو,LY
3,الزن ده هو اكتر حاجة مدمرة للسلام النفسي واحنا بصراحة سلامنا النفسي اهم من كل شيء الحياة قصيرة ومش مستحملة حوارات كتير,EG
4,الكوت كلها ناس بتدور ع علاقات تانيه بس تكون احلي م اللي لسه خالصه اول انبارح,EG


# Train Validation Splitting

In [10]:
df_train, df_valid = train_test_split(df, test_size=0.1, stratify=df['label'], random_state=SEED)

In [12]:
df_train.label.value_counts(), df_valid.label.value_counts()

(EG    46685
 LY    29564
 LB    22369
 SD    11692
 MA    9346 
 Name: label, dtype: int64,
 EG    5187
 LY    3285
 LB    2486
 SD    1299
 MA    1039
 Name: label, dtype: int64)

# Preprocessing

In [13]:
encoder = TextVectorization(max_tokens=50000, ngrams=2)
encoder.adapt(df_train[['tweet']])

In [14]:
VOCAB = encoder.get_vocabulary()

In [15]:
VOCAB_SIZE = len(encoder.get_vocabulary())
VOCAB_SIZE

50000

In [16]:
encoder(df[['tweet']].iloc[0])

<tf.Tensor: shape=(1, 9), dtype=int64, numpy=array([[ 114,    1,  375, 9443,    1,    1,    1,    1,    1]])>

In [17]:
df.tweet.shape

(132952,)

# Modelling

In [18]:
df_train.label.unique().tolist()

['EG', 'LY', 'LB', 'MA', 'SD']

In [19]:
lbl2idx = {k:v for v,k in enumerate(df_train.label.unique().tolist())}
lbl2idx

{'EG': 0, 'LY': 1, 'LB': 2, 'MA': 3, 'SD': 4}

In [20]:
idx2lbl = {k:v for v,k in lbl2idx.items()}
idx2lbl

{0: 'EG', 1: 'LY', 2: 'LB', 3: 'MA', 4: 'SD'}

In [28]:
model0 = Sequential([
    Input(shape=(1,),dtype='string'),
    encoder,
    Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    SimpleRNN(64,),
    Dense(64, activation='relu'),
    Dense(5)
])

In [29]:
model0.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, None, 64)          3200000   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 5)                 325       
                                                                 
Total params: 3,212,741
Trainable params: 3,212,741
Non-trainable params: 0
____________________________________________

In [30]:
model0.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [31]:
history0 = model0.fit(
    x = df_train.tweet,
    y = df_train.label.replace(lbl2idx),
    epochs=4, 
    verbose=1, 
    validation_data = (df_valid.tweet, df_valid.label.replace(lbl2idx)),
    batch_size = 256,
  )

Epoch 1/4
468/468 [==============================] - 84s 173ms/step - loss: 0.9843 - accuracy: 0.6226 - val_loss: 0.6370 - val_accuracy: 0.7767
Epoch 2/4
468/468 [==============================] - 80s 171ms/step - loss: 0.4526 - accuracy: 0.8472 - val_loss: 0.5917 - val_accuracy: 0.7974
Epoch 3/4
468/468 [==============================] - 81s 173ms/step - loss: 0.2228 - accuracy: 0.9261 - val_loss: 0.7818 - val_accuracy: 0.7581
Epoch 4/4
468/468 [==============================] - 80s 172ms/step - loss: 0.0939 - accuracy: 0.9705 - val_loss: 0.9268 - val_accuracy: 0.7708


In [25]:
preds_valid = model0.predict(df_valid.tweet)

416/416 [==============================] - 5s 12ms/step


In [26]:
f1_score(df_valid.label.replace(lbl2idx), np.argmax(preds_valid, axis=-1), average='macro')

0.7575863859999187

# Enhancement

## Bidirectional

In [ ]:
model1 = Sequential([
    Input(shape=(1,),dtype='string'),
    encoder,
    Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    Bidirectional(SimpleRNN(64,)),
    Dense(64, activation='relu'),
    Dense(5)
])

In [ ]:
model1.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [ ]:
history1 = model1.fit(
    x = df_train.tweet,
    y = df_train.label.replace(lbl2idx),
    epochs=4, 
    verbose=1, 
    validation_data = (df_valid.tweet, df_valid.label.replace(lbl2idx)),
    batch_size = 1024,
  )

Epoch 1/4
117/117 [==============================] - 69s 566ms/step - loss: 1.0706 - accuracy: 0.5821 - val_loss: 0.6659 - val_accuracy: 0.7574
Epoch 2/4
117/117 [==============================] - 53s 457ms/step - loss: 0.4881 - accuracy: 0.8311 - val_loss: 0.5512 - val_accuracy: 0.8090
Epoch 3/4
117/117 [==============================] - 53s 449ms/step - loss: 0.2873 - accuracy: 0.9050 - val_loss: 0.5498 - val_accuracy: 0.8132
Epoch 4/4
117/117 [==============================] - 50s 430ms/step - loss: 0.1670 - accuracy: 0.9473 - val_loss: 0.6370 - val_accuracy: 0.8001


In [ ]:
preds_valid = model1.predict(df_valid.tweet)
f1_score(df_valid.label.replace(lbl2idx), np.argmax(preds_valid, axis=-1), average='macro')

416/416 [==============================] - 9s 22ms/step


0.7642087491492553

# LSTM

In [ ]:
model2 = Sequential([
    Input(shape=(1,),dtype='string'),
    encoder,
    Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    Bidirectional(LSTM(64,)),
    Dense(64, activation='relu'),
    Dense(5)
])

In [ ]:
model2.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [ ]:
history2 = model2.fit(
    x = df_train.tweet,
    y = df_train.label.replace(lbl2idx),
    epochs=3, 
    verbose=1, 
    validation_data = (df_valid.tweet, df_valid.label.replace(lbl2idx)),
    batch_size = 512,
  )

Epoch 1/3
234/234 [==============================] - 81s 303ms/step - loss: 0.8862 - accuracy: 0.6650 - val_loss: 0.6206 - val_accuracy: 0.7896
Epoch 2/3
234/234 [==============================] - 54s 231ms/step - loss: 0.3952 - accuracy: 0.8643 - val_loss: 0.5199 - val_accuracy: 0.8242
Epoch 3/3
234/234 [==============================] - 44s 189ms/step - loss: 0.2634 - accuracy: 0.9116 - val_loss: 0.5645 - val_accuracy: 0.8172


In [ ]:
preds_valid = model2.predict(df_valid.tweet)

416/416 [==============================] - 6s 8ms/step


In [ ]:
f1_score(df_valid.label.replace(lbl2idx), np.argmax(preds_valid, axis=-1), average='macro')

0.7801530245421613

### Doubling embedding dimension

In [ ]:
# Let's double the embedding dimension!
model3 = Sequential([
    Input(shape=(1,),dtype='string'),
    encoder,
    Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=64*2,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'),
    Dense(5)
])

In [ ]:
model3.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [ ]:
history3 = model3.fit(
    x = df_train.tweet,
    y = df_train.label.replace(lbl2idx),
    epochs=5, 
    verbose=1, 
    validation_data = (df_valid.tweet, df_valid.label.replace(lbl2idx)),
    batch_size = 1024,
  )

Epoch 1/5
116/116 [==============================] - 56s 398ms/step - loss: 1.0316 - accuracy: 0.6013 - val_loss: 0.6318 - val_accuracy: 0.7782
Epoch 2/5
116/116 [==============================] - 40s 339ms/step - loss: 0.4403 - accuracy: 0.8497 - val_loss: 0.5551 - val_accuracy: 0.8119
Epoch 3/5
116/116 [==============================] - 32s 275ms/step - loss: 0.2753 - accuracy: 0.9086 - val_loss: 0.5460 - val_accuracy: 0.8220
Epoch 4/5
116/116 [==============================] - 32s 274ms/step - loss: 0.1914 - accuracy: 0.9371 - val_loss: 0.6201 - val_accuracy: 0.8163
Epoch 5/5
116/116 [==============================] - 29s 249ms/step - loss: 0.1391 - accuracy: 0.9553 - val_loss: 0.7307 - val_accuracy: 0.8127


In [ ]:
preds_valid = model3.predict(df_valid.tweet)
f1_score(df_valid.label.replace(lbl2idx), np.argmax(preds_valid, axis=-1), average='macro')

462/462 [==============================] - 5s 7ms/step


0.7733267006280761

### Going deeper

![](https://i.kym-cdn.com/photos/images/newsfeed/000/531/557/a88.jpg)


In [ ]:
# Let's double the embedding dimension!
model4 = Sequential([
    Input(shape=(1,),dtype='string'),
    encoder,
    Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=64*2,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    Bidirectional(LSTM(64,  return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(64, activation='relu'),
    Dense(5)
])

In [ ]:
model4.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [ ]:
history4 = model4.fit(
    x = df_train.tweet,
    y = df_train.label.replace(lbl2idx),
    epochs=5, 
    verbose=1, 
    validation_data = (df_valid.tweet, df_valid.label.replace(lbl2idx)),
    batch_size = 256,
  )

Epoch 1/5
462/462 [==============================] - 133s 246ms/step - loss: 0.7308 - accuracy: 0.7272 - val_loss: 0.5192 - val_accuracy: 0.8209
Epoch 2/5
462/462 [==============================] - 71s 154ms/step - loss: 0.3430 - accuracy: 0.8836 - val_loss: 0.4987 - val_accuracy: 0.8272
Epoch 3/5
462/462 [==============================] - 62s 133ms/step - loss: 0.2169 - accuracy: 0.9284 - val_loss: 0.5721 - val_accuracy: 0.8174
Epoch 4/5
462/462 [==============================] - 53s 115ms/step - loss: 0.1499 - accuracy: 0.9502 - val_loss: 0.6483 - val_accuracy: 0.8140
Epoch 5/5
462/462 [==============================] - 51s 111ms/step - loss: 0.1105 - accuracy: 0.9637 - val_loss: 0.7536 - val_accuracy: 0.8098


In [ ]:
preds_valid = model4.predict(df_valid.tweet)
f1_score(df_valid.label.replace(lbl2idx), np.argmax(preds_valid, axis=-1), average='macro')

462/462 [==============================] - 5s 12ms/step


0.771228564206637

### Trying dropout

In [ ]:
# Let's double the embedding dimension!
model5 = Sequential([
    Input(shape=(1,),dtype='string'),
    encoder,
    Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=64*2,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    Bidirectional(LSTM(64,  return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
    Bidirectional(LSTM(32, dropout=0.2, recurrent_dropout=0.2)),
    Dense(64, activation='relu'),
    Dense(5)
])

In [ ]:
model5.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [ ]:
history5 = model5.fit(
    x = df_train.tweet,
    y = df_train.label.replace(lbl2idx),
    epochs=1, 
    verbose=1, 
    validation_data = (df_valid.tweet, df_valid.label.replace(lbl2idx)),
    batch_size = 128,
  )

924/924 [==============================] - 1962s 2s/step - loss: 0.6598 - accuracy: 0.7617 - val_loss: 0.4910 - val_accuracy: 0.8270


In [ ]:
preds_valid = model5.predict(df_valid.tweet)
f1_score(df_valid.label.replace(lbl2idx), np.argmax(preds_valid, axis=-1), average='macro')

462/462 [==============================] - 52s 111ms/step


0.7783550778062927

### Results & Conclusion

1. The embedding layers has much more trainable parameters compared to RNN/LSTM
2. The models suffer from overfitting (Train_f1=0.95 valid_f1=0.7)
3. Changing the embedding dimension didn't improve the f1 score much
4. Stacking another LSTM layer also didn't improve the f1 score much

We'll just use the original model since it's less complex

# Final Model

In [ ]:
final_model = Sequential([
    Input(shape=(1,),dtype='string'),
    encoder,
    Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    Bidirectional(LSTM(64,)),
    Dense(64, activation='relu'),
    Dense(5)
])

In [ ]:
final_model.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

## Retraining on entire train + valid

In [ ]:
df = pd.concat([df_train, df_valid], axis=0).sample(frac=1)

In [ ]:
final_history = final_model.fit(
    x = df.tweet,
    y = df.label.replace(lbl2idx),
    epochs=3, 
    verbose=1, 
    batch_size = 512,
  )

Epoch 1/3
260/260 [==============================] - 82s 270ms/step - loss: 0.8975 - accuracy: 0.6581
Epoch 2/3
260/260 [==============================] - 59s 227ms/step - loss: 0.3941 - accuracy: 0.8651
Epoch 3/3
240/260 [==========================>...] - ETA: 3s - loss: 0.2532 - accuracy: 0.9152

## Assessing on the test set

In [ ]:
df_test = pd.read_csv(test_csv_file_path, lineterminator='\n')

In [ ]:
preds_valid = final_model.predict(df_test.tweet)

416/416 [==============================] - 6s 8ms/step


In [ ]:
f1_score(df_test.label.replace(lbl2idx), np.argmax(preds_valid, axis=-1), average='macro')

0.7801530245421613

## Saving Model

In [ ]:
if using_colab:
  model2.save('bilstm_model')
else:
  model2.save('bilstm_model')

In [ ]:
final_model = Sequential([
    Input(shape=(1,),dtype='string'),
    encoder,
    Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    Bidirectional(LSTM(64,)),
    Dense(64, activation='relu'),
    Dense(5)
])

In [ ]:
final_model.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(
    x = df_train.tweet,
    y = df_train.label.replace(lbl2idx),
    epochs=4, 
    verbose=1, 
    validation_data = (df_valid.tweet, df_valid.label.replace(lbl2idx)),
    batch_size = 1024,
  )

Epoch 1/5
117/117 [==============================] - 162s 1s/step - loss: 1.0895 - accuracy: 0.5724 - val_loss: 0.6490 - val_accuracy: 0.7696
Epoch 2/5
117/117 [==============================] - 155s 1s/step - loss: 0.4881 - accuracy: 0.8337 - val_loss: 0.5248 - val_accuracy: 0.8173
Epoch 3/5
117/117 [==============================] - 147s 1s/step - loss: 0.3303 - accuracy: 0.8879 - val_loss: 0.5333 - val_accuracy: 0.8224
Epoch 4/5
117/117 [==============================] - 151s 1s/step - loss: 0.2513 - accuracy: 0.9152 - val_loss: 0.6042 - val_accuracy: 0.8152
Epoch 5/5
117/117 [==============================] - 143s 1s/step - loss: 0.1984 - accuracy: 0.9327 - val_loss: 0.6644 - val_accuracy: 0.8102


In [ ]:
preds_valid = model.predict(df_valid.tweet)

416/416 [==============================] - 25s 58ms/step


In [ ]:
f1_score(df_valid.label.replace(lbl2idx), np.argmax(preds_valid, axis=-1), average='macro')

0.7749578388615694